In [80]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Necesario para habilitar IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Leer el CSV
data = pd.read_csv('../../data/EstudioCrediticio_TrainP.csv')

### Lectura de datos y borrado columnas id y creditoaprobado

In [81]:
# Eliminar la primera columna
data = data.iloc[:, 1:]

# Eliminar la penúltima columna
data = data.iloc[:, :-2].join(data.iloc[:, -1])

### Obtener variables predictoras y dependiente

In [82]:
# Separar las características (X) y la variable a predecir (y)
X = data.iloc[:, :-1]  # Todas las columnas menos la última
y = data.iloc[:, -1]   # La última columna

### Pipeline con simple imputer

In [83]:
# Identificar columnas numéricas y categóricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline para variables numéricas (Imputación + Escalado)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar con la media
    ('scaler', RobustScaler())
])

# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Crear pipeline final con preprocesamiento y modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

### Pipeline con iterative imputer

In [84]:
# Identificar columnas numéricas y categóricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline para variables numéricas (IterativeImputer + Escalado)
numeric_transformer_iterative = Pipeline(steps=[
    ('imputer', IterativeImputer()),  # Imputación iterativa
    ('scaler', RobustScaler())
])

# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor_iterative = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_iterative, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Crear pipeline final con preprocesamiento y modelo
model_iterative = Pipeline(steps=[
    ('preprocessor', preprocessor_iterative),
    ('regressor', RandomForestRegressor())
])

### Pipeline con PCA

In [85]:
# Identificar columnas numéricas y categóricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

print(len(numeric_cols))

# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline para variables numéricas (Imputación + Escalado + PCA)
numeric_transformer_pca = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputación con la media
    ('scaler', RobustScaler()),
    ('pca', PCA(n_components=16))  # Aplicar PCA con 5 componentes principales
])

# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor_pca = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_pca, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Crear pipeline final con preprocesamiento y modelo
model_pca = Pipeline(steps=[
    ('preprocessor', preprocessor_pca),
    ('regressor', RandomForestRegressor())
])

28


In [86]:
models = {'SimpleImputer': model, 'IterativeImputer': model_iterative, 'PCA': model_pca}

### Conjunto de entrenamiento y prueba

In [87]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Entrenamiento del modelo

In [88]:
for model in models.values():
    model.fit(X_train, y_train)

### Evaluación del modelo

In [89]:
for model in models.keys():
    print(f"Modelo: {model}")
    # Evaluar el modelo
    mae_train = mean_squared_error(y_train, models.get(model).predict(X_train))
    mae_test = mean_squared_error(y_test, models.get(model).predict(X_test))

    print(f"MAE del conjunto de entrenamiento: {mae_train:.2f}")
    print(f"MAE del conjunto de prueba: {mae_test:.2f}")

Modelo: SimpleImputer
MAE del conjunto de entrenamiento: 1.15
MAE del conjunto de prueba: 8.34
Modelo: IterativeImputer
MAE del conjunto de entrenamiento: 1.14
MAE del conjunto de prueba: 8.20
Modelo: PCA
MAE del conjunto de entrenamiento: 3.88
MAE del conjunto de prueba: 27.93
